## Major Projects 20 Questions : Part 1 Classification and Feature Engineering

In [1]:
# Data Source: https://majorprojects.alberta.ca/
# Major Projects are valued at $5 million or greater

## Data Preparation

Import necessary libraries

In [2]:
import pandas as pd
import json
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

Create a Pandas DataFrame from a Excel file

In [3]:
data = pd.read_excel('./Major Projects Export.xlsx')
print(data.shape)
data.head()

(703, 14)


,ProjectId,Name,Estimated Cost,Municipality,Schedule,Sector,Type,Stage,Developer,Contractor,Architect,Project Website,Location,Detail
0,7,StoneGate Landing,3.000000e+09,Calgary,2010 - 2021,Mixed-Use,NaN,Under Construction,WAM Development Group / AIMCo,NaN,NaN,www.stonegatelanding.ca/,"{""type"":""Feature"",""geometry"":{""type"":""Point"",""...",StoneGate Landing is slated to be the largest ...
1,11,Shepard Station Suburban Office Campus Building 1,2.200000e+07,Calgary,2019 - 2020,Commercial,Office: Low-Rise,Under Construction,Shepard Development Corp.,Shepard Development Corporation / CANA / ACE C...,NaN,http://www.shepardstation.com/,"{""type"":""Feature"",""geometry"":{""type"":""Point"",""...",The Shepard Suburban Office Campus is a 33-acr...
2,22,Barron Building Renovation,1.000000e+08,Calgary,Commencing 2021,Residential,Apartment: Mid-Rise,Proposed,Strategic Group,NaN,NaN,https://www.strategicgroup.ca/commercial/calga...,"{""type"":""Feature"",""geometry"":{""type"":""Point"",""...",Conversion of the Barron Building into a 94 un...
3,26,Quarry Crossing II Office Building,7.280000e+07,Calgary,NaN,Commercial,Office: Low-Rise,Proposed,Remington Development Corp.,NaN,NaN,http://www.remingtoncorp.com/leasingLocation.p...,"{""type"":""Feature"",""geometry"":{""type"":""Point"",""...","A 512,000 sq ft office building project in Cal..."
4,32,Nolan Hill TownHomes,5.000000e+06,Calgary,NaN,Residential,Townhouses,Proposed,Jayman Modus,NaN,NaN,NaN,"{""type"":""Feature"",""geometry"":{""type"":""Point"",""...",NaN


Extract from geojson the start location coordinates (Longitude, Latitude)

In [4]:
data['Location'] = data['Location'].apply(json.loads)

Latitude = []
Longitude = []

for i in range(len(data)):
    geo = data['Location'][i]['geometry']
    if geo['type'] == 'Point':
        Longitude.append(geo['coordinates'][0])
        Latitude.append(geo['coordinates'][1])
    elif geo['type'] == 'LineString':
            Longitude.append(geo['coordinates'][0][0])
            Latitude.append(geo['coordinates'][0][1])
    elif geo['geometries'][0]['type'] == 'Point':
            Longitude.append(data['Location'][i]['geometry']['geometries'][0]['coordinates'][0])
            Latitude.append(data['Location'][i]['geometry']['geometries'][0]['coordinates'][1])
    elif geo['geometries'][0]['type'] == 'LineString':
            Longitude.append(data['Location'][i]['geometry']['geometries'][0]['coordinates'][0][0])
            Latitude.append(data['Location'][i]['geometry']['geometries'][0]['coordinates'][0][1])
    else:
        print('New Type Error')
        
data['Start Latitude'] = Latitude
data['Start Longitude'] = Longitude

Drop rows where there is no Estimated Cost and divide cost by a million

In [5]:
data = data.dropna(subset=['Estimated Cost'])
data['Estimated Cost'] = data['Estimated Cost']/(1e6)
data.shape

(695, 16)

Change Schedule Completion, assume 2027 for all proposed Project w/o Schedule

In [6]:
data['Schedule'] = data['Schedule'].str[-4:]
data['Schedule'] = data['Schedule'].fillna(2027)
data['Schedule'] = data['Schedule'].astype(int)

Rename Colums

In [7]:
data.rename(columns={'Name':'Project Name', 'Estimated Cost':'Estimated Cost (millions)', 'Schedule':'Forecasted Completion'}, inplace=True)

Make Type equal to Sector when there is no type

In [8]:
data['Type'] = data['Type'].fillna(data['Sector'])

Drop columns not needed and set ProjectId as index

In [9]:
data.drop(columns=['Contractor','Architect','Project Website', 'Detail', 'Location'], inplace=True)
data.set_index('ProjectId')

,Project Name,Estimated Cost (millions),Municipality,Forecasted Completion,Sector,Type,Stage,Developer,Start Latitude,Start Longitude
ProjectId,,,,,,,,,,
7,StoneGate Landing,3000.0,Calgary,2021,Mixed-Use,Mixed-Use,Under Construction,WAM Development Group / AIMCo,51.172501,-113.975800
11,Shepard Station Suburban Office Campus Building 1,22.0,Calgary,2020,Commercial,Office: Low-Rise,Under Construction,Shepard Development Corp.,50.931721,-113.970596
22,Barron Building Renovation,100.0,Calgary,2021,Residential,Apartment: Mid-Rise,Proposed,Strategic Group,51.046070,-114.076614
26,Quarry Crossing II Office Building,72.8,Calgary,2027,Commercial,Office: Low-Rise,Proposed,Remington Development Corp.,50.966900,-114.002899
32,Nolan Hill TownHomes,5.0,Calgary,2027,Residential,Townhouses,Proposed,Jayman Modus,51.162041,-114.160912
...,...,...,...,...,...,...,...,...,...,...
4268,Ben Calf Robe School Replacement,25.6,Edmonton,2021,Institutional,School,Under Construction,Edmonton Catholic School District,53.571312,-113.438348
4269,École A la Decouverte School,17.2,Edmonton,2022,Institutional,School,Under Construction,The Greater North Central Francophone Educatio...,53.641039,-113.503243
4270,Ecole Secondaire Beaumont Composition High Sch...,14.0,Beaumont,2022,Institutional,School,Under Construction,The Black Gold School Division,53.348926,-113.422593


## Data Exploration

In [10]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ProjectId,695.0,2831.024460,1603.223943,7.000000,776.500000,3760.000000,4049.000000,4272.000000
Estimated Cost (millions),695.0,222.061799,971.862465,5.000000,10.100000,23.000000,73.900000,16000.000000
Forecasted Completion,695.0,2022.105036,3.136504,2014.000000,2020.000000,2021.000000,2024.000000,2027.000000
Start Latitude,695.0,52.510257,1.812646,49.020533,51.044388,52.817036,53.554348,58.483555
Start Longitude,695.0,-113.800780,1.628331,-119.766083,-114.080355,-113.700128,-113.392287,-110.006125


In [11]:
data.Type.value_counts()

Roadwork                79
Other                   63
Apartment: Low-Rise     59
Water/Wastewater        37
Mixed-Use               31
Sports Facility         23
School                  23
Continuing Care         22
Post-Secondary          21
Apartment: High-Rise    21
Solar                   17
Office: Low-Rise        17
Health Care             16
Emergency Services      15
Oil Sands: In Situ      13
Apartment: Mid-Rise     13
Office Renovation       13
Hotel                   12
Transit                 12
Pipelines               11
Flood Mitigation        10
Wind                    10
Telecommunication        9
Arts and Culture         9
Administration           9
Business Park            9
Religious                8
Townhouses               8
Gas                      7
Agriculture              7
Natural Gas              7
Event Space              7
Attractions              7
Upgrader                 6
Park                     5
Shopping Plaza           5
Distribution Centre      5
B